IMPORTING THE LIBRARIES

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"

!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

! pip install --user indic
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))

from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

from indicnlp import loader
loader.load()

from indicnlp.tokenize import indic_tokenize 
import re
import csv

fatal: destination path 'indic_nlp_library' already exists and is not an empty directory.
fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.


HINDI AND ENGLISH TOKENIZERS

In [ ]:
# HINDI AND ENGLISH TOKENIZERS  :::::


tok_hindi= []
tok_english = []

co=1

def tokenize_hindi(text):

  indic_string = text

  res=""

  for text in indic_tokenize.trivial_tokenize(indic_string): 
   
    #print(text)
    text=text.replace('„', r' ')
    text=text.replace('“', r' ')
    text=text.replace('”', r' ')
    text=text.replace('–', r' ')
    text=text.replace('—', r' ')
    text=text.replace('´', r" ")
    text=text.replace('‘', r" ")
    text=text.replace('’', r" ")
    text=text.replace("''", r' ')
    text=text.replace('´´', r' ')
    text=text.replace('…', r' ')
    text=text.replace('.',r' ')
    text=text.replace(',',r' ')
    text=text.replace('-',r' ')
    text=text.replace('"',r' ')
    text=text.replace('|',r' ')
    text=text.replace('।',r' ')
    text=text.replace('?',r' ')
    text=text.replace('@',r' ')
    text=text.replace('_',r' ')
    text=text.replace('♫',r' ')  
    text=text.replace('♪',r' ')
    text=text.replace('!',r' ')
    text=text.replace('#',r' ')
    text=text.replace('$',r' ')
    text=text.replace('%',r' ')
    text=text.replace('^',r' ')
    text=text.replace('&',r' ')
    text=text.replace('*',r' ')
    text=text.replace('(',r' ')
    text=text.replace(')',r' ')
    text=text.replace('/',r' ')
    text=text.replace('{',r' ')
    text=text.replace('~',r' ')
    text=text.replace(':',r' ')
    text=text.replace('[',r' ')
    text=text.replace(']',r' ')
    
    res=res+' '+text

  reaesc = re.compile(r'[a-zA-Z]')
  res = reaesc.sub('',res)
  #reaesc = re.compile(r'[#$%^&*()?,/|}-‚.{~:.]।')---this was just making the special characters as a
  # separate element in the list and hence does not eliminate it from the list and so removed these
  # using the text.replace
  #res = reaesc.sub('',res)

  res = res.replace("''",r' ')
  ### TRIED REMOVING THIS BUT COULD NOT res=res.replace(u'\u200b',r'')
   
  return indic_tokenize.trivial_tokenize(res)


def tokenize_english(l2):
  
  l1 = []
  reaesc = re.compile(r'♫#?[@_!♫♪#$%^&*(.,)<>?/\|}{~:]')
  text = reaesc.sub('',l2)
  text= ''.join(l2)

  #print(text)
  text=text.replace("-"," ")
  text=text.replace("♫"," ")
  text=text.replace("#"," ")
  text=text.replace("?"," ")
  text=text.replace("["," ")
  text=text.replace("@"," ")
  text=text.replace("_"," ")
  text=text.replace("!"," ")
  text=text.replace("♫"," ")
  text=text.replace("♪"," ")
  text=text.replace("%"," ")
  text=text.replace("$"," ")
  text=text.replace("^"," ")
  text=text.replace("&"," ")
  text=text.replace("*"," ")
  text=text.replace("("," ")
  text=text.replace("."," ")
  text=text.replace("."," ")
  text=text.replace(","," ")
  text=text.replace(")"," ")
  text=text.replace("<"," ")
  text=text.replace(">"," ")
  text=text.replace("]"," ")
  text=text.replace(":"," ")
  text=text.replace("~"," ")
  text=text.replace("{"," ")
  text=text.replace("}"," ")
  text=text.replace("|"," ")
  text=text.replace("/"," ")
  text=text.replace("?"," ")
  text=text.replace("fuckin'","fucking")
  text=text.replace("'ii"," will")
  text=text.replace("'ll", r" will")
  text=text.replace("can't", r"can not")
  text=text.replace("won't", r"will not")
  text=text.replace("n't", r" not")
  text=text.replace("'re", r" are")
  text=text.replace("i'm", r"i am")
  text=text.replace("'s" , r" is")
  text=text.replace("'ve", r" have")
  text=text.replace("'ve" , r"have")
  text=text.replace("'d" , r"would")
  text=text.replace("cannot" , r"can not")  
  
  doc = indic_tokenize.trivial_tokenize(text)
  for token in doc:
    reg = re.compile(r'[A-Za-z0-9]')
    if reg.match(token):
      l1.append(token)
  
  length = len(l1)

  list1 = []
  for i in range(0,length-1):
    if l1[i]=='ai' and l1[i+1]=='not':
      l1[i] = "ain't"
      i=i+1
      list1.append(i)
  
  list2 = []

  for i in range(0,len(l1)):
     if i not in list1:
       list2.append(l1[i])
  
  return list2

PARSING THE CSV INTO HINDI AND ENGLISH LISTS

In [ ]:
#!cut -d, -f1 --complement /content/train.csv > /content/new_train.csv 
#statements = open('/content/new_train.csv').read().strip().split('\n')


hindi_list= []
english_list = []


with open('train.csv', 'r') as file:
  reader = csv.reader(file)
  c=0
  res=0
  k=0
  for row in reader:
    m = row[1]
    hindi_list.append(m)
    if "8" in m:
      print(m)
      res=res+1
    if "9" in m:
      q=q+1
    c=c+1
    english_list.append(row[2])      

  print(res)
  print(q)

28 साल पुराना है.
"17 जनवरी, 1989,
100 करोड़ बच्चों के लिए हमें 10 करोड़ मांझी चाहियें -- इस धरती पर उससे कहीं ज्यादा हैं -- एक करोड़ सोल्स, 18 हज़ार करोड़ डॉलर और दस साल.
हम बस के साथ महानगरों 8 खबर कनेक्शन खो दिया है।
उह उह, 20 THOUSA-- 60 - 60,850.
आप जुड़वां टर्बो वी 8 के बारे में बात कर रहे हैं, 560 टट्टू, बेटा बाहर थूकना.
जलवायु परिवर्तन, 2008 का महान वित्तीय संकट दो सरल उदाहरण हैं।
2008 में, एक गूगलर, सुंदर पिचई, एक उद्देश्य पर ले गया अगली पीढ़ी के ग्राहक मंच का निर्माण वेब अनुप्रयोगों के भविष्य के लिए - सर्वश्रेष्ठ ब्राउज़र
अब तुम हमें हमारी 80 देना भव्य पीठ और उसे अपने साथ ले जाओ.
971 01: 47: 13,583
जेजी: या यहाँ एक वीडियो है "क्राश कोर्स"के हिस्से के रूप में: प्रथम विश्व युद्ध के बारेमे (वीडियो) तत्काल कारण ज़ाहिर था ऑस्ट्रियन आर्कड्यूक फ्रांज फर्डिनेंड केसाराजेवो में हत्या, 28जून1914पर,गाव्रिलो प्रिंसिप नाम का एक बोस्नियाई सर्ब राष्ट्रवादी द्वारा
औरत [टीवी पर]: रदरफोर्ड उम्रकैद की सजा काट रहा था... ...एक 1968 शूटिंग के लिए.
एक के आगे 18 जीरो.
हमें पता चला कि कटु बर्ताव लोगों 

In [ ]:
 
      print(m)
      k=k+1

TOKENIZING THE HINDI AND ENGLISH SENTENCES IN THE RESPECTIVE LISTS  

In [ ]:
tok_hindi =  [[None]*400 for _ in range(len(hindi_list))] 
tok_english =  [[None]*400 for _ in range(len(english_list))] 

c=0
for i in range(0,len(hindi_list)):
  ht = tokenize_hindi(hindi_list[i].lower())
  et = tokenize_english(english_list[i].lower())
  if len(ht)>0 and len(et)>0 and len(ht)<=400 and len(et)<=400 and not (len(ht)==1 and ht[0]=='') and not (len(et)==1 and et[0]==''):
    tok_hindi[c]=ht
    tok_english[c]=et
    c+=1
    if c>100:
      break
  

FINAL TOKENIZED LIST ::

In [ ]:
hindi_tokenized = [[] for _ in range(0,c)]
english_tokenized = [[] for _ in range(0,c)]

for el in range(0,c):

  hindi_tokenized[el]=tok_hindi[el]
  english_tokenized[el]=tok_english[el]


VOCABULARY CLASS TO SET THE FREQUENCY COUNT , STRING TO INDEX MAPPING , INDEX TO STRING MAPPING AND VOCAB SIZE

In [ ]:
class vocabulary:
  def __init__(self,tokenized_list):
    self.lists = tokenized_list
    self.vocab_size=0
    self.freq_count={}
    self.stoi={}
    self.itos={}
  
  # This function uses the tokenized_list and then creates the frequency count out of that
  # by using a dictionary which will basically have a  "word:count" mapping
  # hence store the count corresponding to the words
  def get_freq_count(self):
    # self pointer used for referring to the attributes of the class
    for i in self.lists:
      for j in range(0,len(i)):
        if i[j] in self.freq_count:
          self.freq_count[i[j]] = self.freq_count[i[j]]+1
        else:
          self.freq_count[i[j]] = 1

    self.vocab_size = len(self.freq_count.keys())+4
    # The vocab size 4 larger than the freq_count dictionary since we will also have
    # unk , ss , es and pad in our vocab.
    return self.freq_count,len(self.freq_count.keys())+4

  #This function stores the string to index mapping for each of the words in the dictionary freq_count
  def string_to_index(self):
    st=0
    self.stoi["<unk>"]=0
    self.stoi["<ss>"]=1
    self.stoi["<es>"]=2
    self.stoi["<pad>"]=3
    st=4
    for k,v in self.freq_count.items():
      self.stoi[k]=st
      st+=1
    #return self.stoi

  #This function stores the index to string mapping for each of the words from the already set mapping of stoi
  def index_to_string(self):
    for k,v in self.stoi.items():
      self.itos[v]=k

    #return self.itos

  # This function returns the vocab size 
  def vocab_size(self):

    a,b = get_freq_count(self.lists)
    self.vocab_size =b

    #return b

  

In [ ]:
# Creating the object for the vocabulary class corresponding to the hindi language by passing
# the tokenized list of hindi words 
h_obj = vocabulary(hindi_tokenized)
h_obj.get_freq_count()
h_obj.string_to_index()
h_obj.index_to_string()

# Similarly creating the object for the vocabulary class corresponding to the english language
e_obj = vocabulary(english_tokenized)
e_obj.get_freq_count()
e_obj.string_to_index()
e_obj.index_to_string()

In [ ]:
# Obtaining the string to index mapping for the hindi and english language 
h_stoi = h_obj.stoi
h_itos = h_obj.itos
e_stoi = e_obj.stoi
e_itos = e_obj.itos

CREATING BATCHES OF TOKENIZED SENTENCES ::

In [ ]:
# Helper function used inside the create_batches() function which takes in a tokenized list and 
# return the same list just in place of a word we have the index corresponding to it
def tokenizedlist_to_indexes(tokenized_list,stoi_dict):
  final_indices = [[] for _ in range(0,c)]
  for st in range(0,len(tokenized_list)):
    llist = tokenized_list[st]
    h1 = []
    for j in range(0,len(llist)):
      ele =  llist[j]
      h1.append(stoi_dict[ele])
    final_indices[st]=h1

  return final_indices

In [ ]:
# Helper function that helps computing the max sentence length per batch so that it is easy
# to make the sentences of similar length by padding and then combine them into same batch 
def max_length_per_batch(hindi_english_pair_list):
  max_length_of_hindi_batch = []
  max_length_of_english_batch = []
  count=0
  indx=0
  maxa=0
  maxb=0
  a=0
  b=0
  for el in range(0,len(hindi_english_pair_list)):
    count+=1
    value = hindi_english_pair_list[el]
    a = len(value[0])
    b = len(value[1])
    maxa = max(a,maxa)
    maxb = max(b,maxb)
    if count==batch_size:
      max_length_of_hindi_batch.append(maxa)
      max_length_of_english_batch.append(maxb)
      maxa=0
      maxb=0
      a=0
      b=0
      count=0
  
  if maxa!=0:
    max_length_of_hindi_batch.append(maxa)
    max_length_of_english_batch.append(maxb)

  return max_length_of_hindi_batch,max_length_of_english_batch

In [ ]:
# This function takes in the hindi and the english tokenized list along with the batch_size and h_stoi and e_stoi
# This fn is mainly used for distributing the hindi and english sentences into batches of size =  batch_size
# The batches created are such that they have all the sentences of similar length and if not , then of as
# equal length as possible else we will apply the "<pad>" token to make all these of equal length

def create_batches(hindi_tokenized,english_tokenized,batch_size,h_stoi,e_stoi):
  
  # This list will store the pairs of hindi and english sentence along with the length of hindi sentence which
  # will be used as a criteria for dividing of sentences into batches.
  hindi_english_pair_list = []

  # CONVERT THE LIST OF TOKENS OF HINDI TO THE RESPECTIVE INDICES

  hindi_indices = tokenizedlist_to_indexes(hindi_tokenized,h_stoi)
 
  # CONVERT THE LIST OF TOKENS OF ENGLISH TO THE RESPECTIVE INDICES

  english_indices = tokenizedlist_to_indexes(english_tokenized,e_stoi)

  # hindi_english_pair_list will have the hindi_indices,english_indices and length of hindi_sentence
  for el in range(0,len(hindi_indices)):
    pairs = []
    pairs.append(hindi_indices[el])
    pairs.append(english_indices[el])
    pairs.append(len(hindi_indices[el]))
    hindi_english_pair_list.append(pairs)

  # sort the list on the basis of the 3rd parameter ,i.e. the length of the hindi_sentence
  # This is done so that now batches of similar length sentences can be grouped with each other
  hindi_english_pair_list.sort(key=lambda x: x[2])

  # computing max sentence length corresponding to each batch

  max_length_of_hindi_batch, max_length_of_english_batch = max_length_per_batch(hindi_english_pair_list)

  # Number of batches to be made each of size batch_size

  no_of_batches = len(max_length_of_hindi_batch)

  hindi_batches = [[] for _ in range(0,no_of_batches)]
  english_batches = [[] for _ in range(0,no_of_batches)]

  # last batch  which may be having lesser no of sentences than batch_size
  last_batch_size = len(hindi_indices)%batch_size
  

  # Now , here for each of the values we will divide them into baches of size batch_size and length
  # of each sentence will be max_length_of_english/hindi_batch + 2 since we will be adding a <ss> and
  # <es> i.e. start of sentence and end of sentence token to each of them.
  # For all those sentences of the batch , whose size is less than that of the largest sentence
  # we append a  <pad> to it to make all sentences of one batch of equal size
  count = 0
  v=0
  while v<len(hindi_english_pair_list):
    m_len = max_length_of_hindi_batch[count]
    single_batch = [[None]*(m_len+2) for _ in range(0,batch_size)]
    for m in range(v,min(v+batch_size,len(hindi_english_pair_list))):
      sent = hindi_english_pair_list[m][0]
     # add padding  <pad>
      while len(sent)<m_len :
        sent.append(3)
    # add <ss>
      sent.insert(0,1)
    # add <es>
      sent.append(2)
    # add this sentence to its batch
      single_batch[m-v]=sent
    
    if count==len(max_length_of_hindi_batch)-1:
      single_batch = single_batch[:last_batch_size]
   
    # converting each of the batches into a tensor and then converting it such that each column corresponds
    # to one sentence mapping
    single_batch_tensor =  torch.transpose(torch.tensor(single_batch),0,1)
    
    # The final hindi_batches list that contains the 
    hindi_batches[count]=single_batch_tensor
    v=v+batch_size
    count+=1

# Repeat the same above thing for the english sentences i.e. to make english_batches
  count = 0
  v=0
  while v<len(hindi_english_pair_list):
    m_len = max_length_of_english_batch[count]
    single_batch = [[None]*(m_len+2) for _ in range(0,batch_size)]
    for m in range(v,min(v+batch_size,len(hindi_english_pair_list))):
      sent = hindi_english_pair_list[m][1]
      while len(sent)<m_len :
        sent.append(3)
      sent.insert(0,1)
      sent.append(2)
      single_batch[m-v]=sent
    
    if count==len(max_length_of_english_batch)-1:
      single_batch = single_batch[:last_batch_size]

    single_batch_tensor =  torch.transpose(torch.tensor(single_batch),0,1)
    
    english_batches[count]=single_batch_tensor
    v=v+batch_size
    count+=1

  return hindi_batches,english_batches



In [ ]:
batch_size = 32

l1, l2  = (create_batches(hindi_tokenized,english_tokenized,batch_size,h_stoi,e_stoi))


ENCODER CLASS WITH BI-GRU : 

In [ ]:
class Encode(nn.Module):
  def __init__(self, e_input, e_embedding,
                           e_hidden_size, num_layers):
    super(Encode, self).__init__()

    # Number of layers in the GRU
    self.num_layers = num_layers

    # number of features for GRU to remember
    self.hidden_size = e_hidden_size

    # No of final nodes to drop in order to avoid overfitting
    self.dropout = nn.Dropout(0.5)

    # applying a embedding on the given input_size i.e. the hindi vocabulary 
    self.embedding = nn.Embedding(e_input, e_embedding , padding_idx=3)
    
    #Applies a multi-layer GRU to an input sequence.

    self.GRU = nn.GRU(e_embedding,e_hidden_size,2*num_layers, dropout = 0.5 , bidirectional=True)

  # Dimensions of input [hindi_sentence_length, batch_size](Every sentence of the batch will have the same size since padding was used to make them of equal size)
  def forward(self, input):
    
    # Dimensions of embedding[hindi_sentence_length  , batch_size , embedding_size]
    # Here , the words of the sentence x for all the sentences of the batch size 
    # will be learning an embedding which was initially created using the constructor and function above
    # i.e. nn.Embedding 
    embedding1 = self.embedding(input)
                
    # Avoid overfitting by dropping some of the nodes
    embedding = self.dropout(embedding1)
    
   
    # The embedding is then passed through the GRU layer
    #Dimensions of output [hindi_sentence_length , batch_size , hidden_size*2]
    #Dimensions of hidden state [num_layers*2, batch_size, hidden_size]
    encoder_out , hidden_state = self.GRU(embedding)
    #print(encoder_out.size())

    hidden_final =torch.zeros(num_layers,input.shape[1],hidden_state.shape[2]*2).to(device)

    # THIS IS DONE SINCE WHAT WE GET AS OUTPUT FROM A BI-GRU IS SOMETHING LIKE THIS ::
    # [FORWARD1,BACKWARD1,FORWARD2,BACKWARD2............]
    # SO  CONCATENATED THE OUTPUTS OF THE FORWARD AND BACKWARD OF EACH OF THE 2 CONSECUTIVE HIDDEN LAYERS 
    # AND THEN FED THE RESULTANT AS THE HIDDEN LAYER OUTPUT TO THE DECODER.
    for i in range(0,(hidden_final.shape[0])):
      hidden_final[i]=torch.cat((hidden_state[i*2,:,:],hidden_state[i*2+1,:,:]),dim=1)
    

    return encoder_out , hidden_final


ATTENTION CLASS :: 

In [ ]:
class Attention(nn.Module):
  def __init__(self,e_hidden_size,d_hidden_size):

    super(Attention, self).__init__()

    self.attention  =  nn.Linear((2*e_hidden_size) + d_hidden_size , d_hidden_size)
    self.enout = nn.Linear(2*e_hidden_size,d_hidden_size)
    self.dhid  = nn.Linear(d_hidden_size,d_hidden_size)
    self.del_d_hidden = nn.Linear(d_hidden_size, 1)

  def forward(self,hidden,e_out,batch_size,hsentence_length):

    # We need to repeat the hidden layer of decoder "hsentence_length" number of times
    # This is because the e_out have the dimension of [hsentence_length,batch_size,hidden_dimension*2(bi)]
    # and since when we are trying to find attention we are searching for all words of hsentence_length and 
    # the result we want after applying the operations, i.e. the final attention vector to be of size
    # hsentence_length and hence , we need to repeat the hidden layer of decoder "hsentence_length"
    # no of times.
    # Initially-- #hidden = [num_of_layers,batch_size,d_hidden_size]
        #e_out = [hsentence_length, batch_size, e_hidden_size]
    #print("hidden1->", hidden.size())
    #print("eout1->",e_out.size()) 
    '''
    hidden1-> torch.Size([32, 1024])
    eout1-> torch.Size([3, 32, 1024])
    here1 -- torch.Size([32, 3, 1024]) torch.Size([32, 3, 1024])
    here -- torch.Size([32, 3, 1024])
    hidden2-> torch.Size([32, 3, 1024])
    eout2-> torch.Size([32, 3, 1024])
    '''

    
    hidden = hidden[:,None] 

    hidden = torch.repeat_interleave(hidden,repeats=hsentence_length,dim=1)
        
    #print("here1 --", hidden.size() , hidden.squeeze(dim=0).size())


    #hidden= hidden.squeeze(dim=0)

    #print("here --", hidden.size())
    hidden = hidden.permute(0,1,2)

    e_out = e_out.permute(1, 0, 2)
    
    # Now :: #hidden = [batch_size,hsentence_length,d_hidden_size]
        #e_out = [batch_size,hsentence_length , e_hidden_size ]

    # Now, applying the attention to the hidden of decoder and e_out by first concatenating them in the 3rd
    # dimension and then applying an activation to it.
    #print("hidden2->", hidden.size())
    #print("eout2->",e_out.size())

    concatenated_tensor = torch.cat((hidden, e_out), dim = 2)
    
    concatenated_tensor  = self.attention(concatenated_tensor)

    #c2 = (self.enout(e_out)  + self.dhid(hidden))

    #print("c2->" ,c2.size())

    #print("concatenated->" , concatenated_tensor.size())
    #print(concatenated_tensor.size())  #[32, 3, 1024]
    attn_tensor = torch.tanh(concatenated_tensor)

    #print("attn=>" , attn_tensor.size())
    #attn_tensor = [batch_size,hsentence_length, d_hidden_size]
    
    attention = self.del_d_hidden(attn_tensor) # [batch size, hsentence_length , 1] 
    
    #print("attention=>" , attention.size())

    attention = attention.squeeze(dim=2)
    #[batch size, hsentence_length]
    #attention= [batch size,hsentence_length]

    attention_resultant = F.softmax(attention, dim=1)
    
    #print("result->" ,attention_resultant.size())


    '''
    attn=> torch.Size([32, 3, 1024])
    attention=> torch.Size([32, 3, 1])
    result-> torch.Size([32, 3])
    '''
    # This will give a attention vector wherein for every statement present in the batch 
    # the attention vector corresponding to it will contain what is the importance of each word
    # of the sentence for the prediction of this new word so the softmax will convert the value corresponding
    # to each word of the sentence in b/w 0 and 1 , such that the sum of importance of all the words
    # of a sentence is equal to 1.

    return attention_resultant




DECODER CLASS WITH GRU::

In [ ]:
class Decode(nn.Module):
  def __init__(self, d_input, d_embedding,hidden_size, num_layers,result_size,attn,e_hidden_size):
    
    super(Decode, self).__init__()


    # Number of layers in the lstm
    self.num_layers = num_layers
    
    # Output size of the word embedding NN
    self.embedding_size = d_embedding

    # Set the attention vector 
    self.attn = attn

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size


    # The result which is obtained after the LSTM model results are passed through
    # a fully connected layer to give the final vector which contains a probability
    # corresponding to each word in the vocabulary.
    #Set the final result size to be obtained 
    self.result_size = result_size

    # Regularization parameter
    self.dropout = nn.Dropout(0.5)

    # Shape (hindi_vocab_size,embedding_size) [input size, embedding dims]
    #The module that allows you to use Embedding is torch.nn.Embedding.
    #It takes two parameters : the vocabulary size and the dimensionality of the embedding
    self.embedding = nn.Embedding(d_input, d_embedding)

    # Shape(embedding_size,no_of_layers_of_lstm,hidden_size) [embedding dims, hidden size, num layers]
    self.GRU = nn.GRU(d_embedding + 2*e_hidden_size, hidden_size, num_layers, dropout = 0.5 , bidirectional=False)
                             # This 1024 is for the 2*e_hidden_size


    # Tried putting 1/4 to 1/2 of bias length=1
    # in order to LSTM to remember more by default but that did not improve the results on the validation set.
    # Similar to the earlier case of encoders
    #m = self.e_LSTM._all_weights
    #print(self.e_LSTM.bias_ih_l0.data[0:0])
    #self.e_LSTM.bias_ih_l0.data[20:40]=1
    #self.e_LSTM.bias_ih_l1.data[20:40]=1
    #self.e_LSTM.bias_hh_l0.data[20:40]=1
    #self.e_LSTM.bias_hh_l1.data[20:40]=1


    # Shape(hidden_size,english_vocab_size) [embedding dims, hidden size, num layers]
    #For fully connected layer, number of input features = number of hidden units in LSTM. 
    self.fully_connected = nn.Linear(2*e_hidden_size + hidden_size + d_embedding, result_size)

    self.softmax = nn.Softmax(dim=1)
  # Shape of x (32) [batch_size]
  def forward(self,output_from_prev, hidden_state , en_out ):

    # Shape of x (1, 32) [1, batch_size]
    output_from_prev = output_from_prev.unsqueeze(0)

    hidden_state1 = hidden_state[-1]
    # Shape(1, 32, embedding_Size) [1, batch_size, embedding dims]
    embedding1 = self.dropout(self.embedding(output_from_prev))

    #Applying the activation function to the embedding
    embedding = F.relu(embedding1)
    #print("passed hidden state-->" ,hidden_state1.size())
    # Passing the encoder outputs (i.e. the hidden states from the encoder) and the decoder's prev layer hidden
    # unit to the attention function to get the value of the attention vector.
    # from the attention function we get the value as [batch_size,en_out.shape[0]i.e. the length of the input hindi statement]
    attention_value = self.attn(hidden_state1,en_out,embedding1.shape[1],en_out.shape[0])
   #    attention_value = self.attn(hidden_state,en_out,'output_from_prev.shape[1]'32,en_out.shape[0])

    # Now we need to use this attention to create a weighted vector , which will be having the importance 
    # corresponding to each word 
    # So on multiplying the attention_value with the en_out i.e. the hidden layers of the encoder we will
    # get for each hidden layer the importance of every word for its contribution to predict the 
    # next word

    # attention_value -> [batch_size,input_hindi_length]
    # en_out -> [input_hindi_length, batch_size,e_hidden_size]
    # now in order to get the weighted input whose dimension should be appropriate to be fed to the 
    # GRU layer i.e. it should be of the dimensions ->[1,batch_size,embedding_size+weighted_vector concatenated to it]
    # so attention value from batch_size,input_hindi_length ----> [batch_size,1,input_hindi_length] and 
     
    attention_value  = attention_value.unsqueeze(dim=1)

    # en_out -> [batch_size , input_hindi_length ,e_hidden_size]

    en_out  = en_out.permute(1,0,2)

    # then multiplying both of them to give the weighted vector -> [batch_size,1,e_hidden_size]
    
    #weighted = torch.bmm(a, encoder_outputs)

    weighted_tensor = torch.matmul(attention_value,en_out)

    # and then concatenating the embedding[1,batch_size,embedding_size] and the weighted vector [batch_size,1,e_hidden_size] (first permute to -> [1,batch_size,e_hidden_size])
    weighted_tensor = weighted_tensor.permute(1,0,2)
    #print(weighted_tensor.size())
    #print(embedding.size())
    concats = torch.cat((embedding,weighted_tensor),dim=2)
   

    # and then feeding as input to the GRU  
    #print("concts->" ,concats.size()) # 1,32,1324
    #print("hiiiddd->" ,hidden_state.size(), hidden_state.unsqueeze(0).size()) # 32,1024 and 1,32,1024
  #################
    results, hidden_state = self.GRU(concats,hidden_state)

    # Shape of outputs (1, 32, hidden_size) [1, batch_size , hidden_size]
    # Shape(no_of_layers_in_lstm, 32 ,hidden_size)[num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
   # results, hidden_state = self.GRU(embedding, hidden_state)

    #results = results[:, :, :2*self.hidden_size] + results[:, :, 2*self.hidden_size:]

    # Shape of predictions (1, 32, 4556) [ 1, batch_size , output_size]
    #Before putting the lstm output into fc layer it has to be flattened out as done above using
    # nn.Linear
     #Output Size =  1*32*english_vocab_size -> as for each word in all the 64 batches simulatenously
    # processing , there would be a prob associated with every word present in the vocab and the word with 
    # max prob is choosen as the value to be fed to the next LSTM input to 1st layer (this is done later on

    # in the training step)
    ##########################          prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))

    english_pred = self.fully_connected(torch.cat((results,weighted_tensor,embedding),dim=2))
    
    
    # Shape --> predictions [batch_size , output_size]
    english_pred = english_pred.squeeze(0)
    #print("early==>",english_pred)
    english_pred =  self.softmax(english_pred)
    #print("last==>",english_pred)
    return english_pred, hidden_state



SETTING THE HYPERPARAMETERS FOR THE ENCODER AND DECODER::

In [ ]:
batch_size=32
input_size_encoder = h_obj.vocab_size
encoder_embedding_size = 100
e_hidden_size = 256
num_layers = 2
encoder_gru = Encode(input_size_encoder, encoder_embedding_size,
                           e_hidden_size, num_layers).to(device)
#print(encoder_gru)
input_size_decoder = e_obj.vocab_size
decoder_embedding_size = 100
d_hidden_size = 512
num_layers = 2
output_size = e_obj.vocab_size
teacher_forcing_ratio = 0.5
attention  = Attention(e_hidden_size,d_hidden_size)
decoder_gru = Decode(input_size_decoder, decoder_embedding_size,
                           d_hidden_size, num_layers, output_size,attention,e_hidden_size).to(device)
#print(decoder_gru)
# THE SIZE OF THE INPUT TO THE DECODER and size of the result size would be same as the input to
# the decoder is one word from each of the batch that is fed to the system of lstm simultaneously and the output
# would be 1 word for all the 32 batches.
learning_rate = 0.001

# SGD Did not perform good for both the encoders and the decoders and Adam gave the best possible performance.
#e_opt = optim.SGD(encoder_lstm.parameters(), lr=learning_rate)
e_opt = optim.Adam(encoder_gru.parameters(), lr=learning_rate)
#d_opt = optim.SGD(decoder_lstm.parameters(), lr=learning_rate)
d_opt = optim.Adam(decoder_gru.parameters(), lr=learning_rate)

OBTAINING THE BEST SET OF PREDICTIONS USING THE SEQ2SEQ MODEL (Helper function for the predictions function which predicts the english output for given hindi input)

In [ ]:
def get_best_words(hind_list,largest_sentence,device):
  eng_sentence = ""
  input_sentence1 = torch.tensor(hind_list).resize_(len(hind_list),1).to(device)

  with torch.no_grad():
    e_outs, hidden = encoder_gru.forward(input_sentence1)
    
    outputs = [1]
   # We append a <sos> token to the start of the string and then get the output from the decoder lstm and then 
   # get the outputs in the output till we don't encounter a <eos> token.
   # At the end we will get the index of the best prediction set of words.
    for p in range(largest_sentence):
      #last_output_word 
      last_out = outputs[-1]

      with torch.no_grad():
        #print("hhs->" ,hidden.size())
        output, hidden= decoder_gru.forward(torch.LongTensor([last_out]).to(device),hidden,e_outs)
        bg1,bg2 = torch.max(output, 1)
        #print("actual out",bg1 , bg2)
        highest_prob_word = bg2.item()
        
      if highest_prob_word!=2:
        outputs.append(highest_prob_word)
        eng_sentence += e_itos[highest_prob_word]
        eng_sentence += " "
      else:
        break

  return eng_sentence

In [ ]:
def predictions(input_sentence, hindi, english, device, largest_sentence=500):
     
    hind_list  = tokenize_hindi(input_sentence)
    hind_list.insert(0,"<ss>")
    hind_list.append("<es>")
    t_indx = []
    for word in hind_list:
      if word in h_stoi:
	      t_indx.append(h_stoi[word])

    # Tokenized the hindi sentence obtained from the dev set .. appended sos and eos token at the start and end of the token list
    # and then converted the set of words in the tokens to the corresponding indices and hence we get the sentence to be fed as input
    # to the encoder ...which on receiving the input , using the GRU return the hidden state , which when fed to the decoder ,gives the
    # required output from which the maximum probability word is choosen.
    # At the end the final english sentence is generated and returned.
       
    final_sentence = get_best_words(t_indx,largest_sentence,device)


    if(final_sentence[-1]==' '):
      final_sentence = final_sentence[:-1]
    if(len(final_sentence)>1):
      final_sentence = final_sentence[0].upper()+final_sentence[1:]
    if(input_sentence[-1] in ['.','!','?']):
      final_sentence = final_sentence + input_sentence[-1]
      
    return final_sentence

BACK PROPAGATION ::

In [ ]:
def bptt(set_of_outs,english_sent):
   # Clear the accumulating gradients
    e_opt.zero_grad()
    d_opt.zero_grad()
    # Calculate the loss value for every epoch
    # The loss function is such that it does not take into the set_of_outs and english_sent as it is 
    # but we need to convert their dimensionality

    # This is done to remove the <sos> token i.e. the first token
    # Calculate the gradients for weights & biases using back-propagation
    loss = F.NLLLoss(set_of_outs[1:].reshape(-1,e_obj.vocab_size),
                            english_sent[1:].reshape(-1), ignore_index=3)
    
    loss.backward()

    # Clip the gradient value is it exceeds > 0.5 -> this is done to avoid the exploding gradient problem
    # which sometimes starts giving NaN as the loss value
    torch.nn.utils.clip_grad_norm_(encoder_gru.parameters(), max_norm=0.8)
    torch.nn.utils.clip_grad_norm_(decoder_gru.parameters(), max_norm=0.8)
    # Update the weights values using the gradients we calculated using backpropagation 
    e_opt.step()
    d_opt.step()

    return loss.item()

TRAINING THE MODEL ::

In [ ]:
import torch.nn.functional as F
# Training the model for 20 epochs , by taking batches from the training set iterator made and then
# sending each of the sentences of the batch one by one as input to the encoder and then receiving
# the results from decoder appending it to a set_of_outs array , comparing with the original sentence
# and then backpropagating the loss.

hindi_input= "लानत है तुम पर!"
# Damn you!

for q in range(0,10):
  print("Epoch Number -> {} / {}".format(q+1, 20))
  #eng_output = predictions(hindi_input, l1, l2, device,largest_sentence=400)
  eng_output = predictions(hindi_input, l1, l2, device,largest_sentence=400)
  print("RESULT:: " , eng_output)
  break
  #for every batch present 
  for index in range(0,len(l1)):
    
    # The hindi_sent and english_sent are the batches of sentences obtained from the respective hindi and english lists
    hindi_sent = l1[index].to(device)
    english_sent = l2[index].to(device)

    # The hindi_sent is then passed through the encoder to get the hidden state which will then be 
    # forwarded to the decoder

    en_outs , hidden_state = encoder_gru.forward(hindi_sent)
    
    # SOS TOKEN TO BE FIRST FED TO THE GRU 
    in_val = english_sent[0]
    
    # The final set_of_outs vecor which will be having the indexes of the words which will occur with maximum probability 
    set_of_outs = torch.zeros(english_sent.shape[0],english_sent.shape[1], e_obj.vocab_size).to(device)
    
    # The loop is iterated for the english_sentence length
    for k in range(1,english_sent.shape[0]):

        # The decoder provides the best predicted set of words in the output and also the hidden_state
        # which is then passed to the next one as an input
        output, hidden_state = decoder_gru.forward(in_val, hidden_state , en_outs)

        set_of_outs[k] = output
        
        #0th dimension is batch size, 1st dimension is word embedding
        #print(the word embedding having the highest probability with respect to the current word and hence
        # get the index corresponding to that word)
        bg2 = output.argmax(1) 
        # Either pass the next word correctly from the dataset or use the earlier predicted word
        if random.random() < teacher_forcing_ratio:
          in_val = english_sent[k]
        else : 
          in_val = bg2

    # Perform the backpropagation to backpropagate the loss value and update the weights 
    loss_val = bptt(set_of_outs,english_sent)
    #print("end=======================================================")
  print("Loss in each epoch -> {}".format(loss_val))
  

Epoch Number -> 1 / 20
RESULT::  When you new painting hill law applies admirable subtle unionseek phenomena carina another pequod fighting new <unk> global right decades will pequod basics phenomena whom at web rick black ticket thank pequod alive profession coming eddie fear on find when physically at phenomena regular pay will pequod meter <unk> scott what week pequod eyes too reasons gonna subtle british decks face program bandaged flu difference at gonna including profession at muslims pequod draw british more back fear worse when boundaries side pequod black alive alive meter truth yb completely costs week phenomena have evolves going regular you like black mean teleportation could d profession that going liked thoughts interviews push snare gonna yet information meter know stop new fighting its stolen feel law throughout alive fellow easier 111 alive push smaller decks easier a meter program know ideal rick you through alive pequod ideal that celled here thank physically power p

GETTING THE RESULTS ON DEV SET ::

In [ ]:
f1=open("answer.txt","a")
with open('hindistatements.csv', 'r') as file:
  reader = csv.reader(file)
  for row in reader:
    hindi_word = row[2]
    res1 = predictions(hindi_word, l1, l2, device,largest_sentence=500)
    f1.write(res1)
    f1.write("\n")
f1.close()


FileNotFoundError: ignored

In [ ]:
import math
def beam_search_decoder1(output_tensor, beam_width):
  result_list = []
  result_list.append([[], 0])
  #print(type(result_list))
  for ind in range(0,len(output_tensor)):
    list_per_word = []
		# expand each current candidate
    for i in range(0,len(result_list)):
      seq= result_list[i][0]
      prob_value = result_list[i][1]
      for j in range(0, len(output_tensor[ind])):
        l1 = result_list[i][0]
        #print(type(output_tensor[ind][j]) , output_tensor[ind][j])
        s1 = result_list[i][1] + (0-math.log(output_tensor[ind][j]))
        ind_with_probval = (l1 + [j],s1)
        list_per_word.append(ind_with_probval)
      list_per_word.sort(key=lambda x:x[1])
      list_per_word = list_per_word[0:beam_width]
    result_list = list_per_word
    
  return result_list


In [ ]:
def get_best_words_beam(hind_list,largest_sentence,device):
  eng_sentence = ""
  input_sentence1 = torch.tensor(hind_list).resize_(len(hind_list),1).to(device)

  with torch.no_grad():
    e_outs, hidden = encoder_gru.forward(input_sentence1)
    
    outputs = [1]
    prob_per_word = torch.empty(0,e_obj.vocab_size)
   # We append a <sos> token to the start of the string and then get the output from the decoder lstm and then 
   # get the outputs in the output till we don't encounter a <eos> token.
   # At the end we will get the index of the best prediction set of words.
    for p in range(largest_sentence):
      #last_output_word 
      last_out = outputs[-1]

      with torch.no_grad():
        #print("hhs->" ,hidden.size())
        output, hidden= decoder_gru.forward(torch.LongTensor([last_out]).to(device),hidden,e_outs)
        bg1,bg2 = torch.max(output, 1)
        highest_prob_word = bg2.item()
        
        
      if highest_prob_word!=2:
        outputs.append(highest_prob_word)
        print(output)
        prob_per_word = torch.cat((prob_per_word, output),0) 
        #eng_sentence += e_itos[highest_prob_word]
        #eng_sentence += " "
      else:
        break

    print(prob_per_word.size())
    print(len(outputs))

    beam_search_decoder1(prob_per_word,3)
  return eng_sentence

In [ ]:
def predictions_beam_search(input_sentence, hindi, english, device, largest_sentence=500):
     
    hind_list  = tokenize_hindi(input_sentence)
    hind_list.insert(0,"<ss>")
    hind_list.append("<es>")
    t_indx = []
    for word in hind_list:
      if word in h_stoi:
	      t_indx.append(h_stoi[word])

    # Tokenized the hindi sentence obtained from the dev set .. appended sos and eos token at the start and end of the token list
    # and then converted the set of words in the tokens to the corresponding indices and hence we get the sentence to be fed as input
    # to the encoder ...which on receiving the input , using the GRU return the hidden state , which when fed to the decoder ,gives the
    # required output from which the maximum probability word is choosen.
    # At the end the final english sentence is generated and returned.
       
    final_sentence = get_best_words_beam(t_indx,largest_sentence,device)
    print(final_sentence)

    if(final_sentence[-1]==' '):
      final_sentence = final_sentence[:-1]
    if(len(final_sentence)>1):
      final_sentence = final_sentence[0].upper()+final_sentence[1:]
    if(input_sentence[-1] in ['.','!','?']):
      final_sentence = final_sentence + input_sentence[-1]
      
    return final_sentence